In [1]:
(require circuitous
         esterel-calculus/circuits/compiler
         redex/reduction-semantics)

Some possibly helpful laws:

- Posulate 1: `¬( GO ∨ (RES ∧ SEL))` implies all `E'` is `0` and all `kn` are `0`.
- - Correlary 1: This holds for `¬(GO ∨ SEL)`
- - Correlary 2: This holds for `¬(GO ∨ RES)`
- Postulate 2: forall `n` `m`, `m != n`, `¬ (kn ^ km)`, that is at most one `k` wire is selected. (Note: This might be wrong, if GO is given to a selected and resumed pause... this should not be allowed but im not sure...)


In [2]:
(define p
  (circuit
   #:inputs (p_SEL p_K0 p_Kn)
   #:outputs (SEL K0 Kn)
   (SEL = p_SEL)
   (K0 = p_K0)
   (Kn = p_Kn)))

In [3]:
(define start
  (circuit
   #:inputs (GO S RES p_SEL p_K0 p_Kn q_SEL q_K0 q_Kn)
   #:outputs (SEL K0 Kn)
   ;; p
   (p_GO = (and GO S))
   (p_RES = RES)
   ;; these wires are not needed for the proof, and are never read so we can 
   ;; ignore them
   ;(p_SUSP = SUSP)
   ;(p_KILL = KILL)
   ;; q
   (q_GO = (and GO (not S)))
   (q_RES = RES)
   ;; these wires are not needed for the proof, and are never read so we can 
   ;; ignore them
   ;(q_SUSP = SUSP)
   ;(q_KILL = KILL)
   ;; out
   (SEL = (or p_SEL q_SEL))
   (K0 = (or p_K0 q_K0))
   (Kn = (or p_Kn q_Kn))))

(assert-same start 
              (rename
               #:constraints #f
               (compile-esterel (term (present Ss (nts pp 1) (nts q 1))))
               '[Ss S]
               '[pp-SEL p_SEL]
               '[q-GO q_GO]
               '[q-SEL q_SEL]
               '[ppK0 p_K0]
               '[qK0 q_K0]
               '[ppK1 p_Kn]
               '[qK1 q_Kn]
               '[K1 Kn]))
        

subprocess: `execute' access denied for /Users/florence/playground/rosette/rosette/solver/smt/../../../bin/z3
  context...:
   /Users/florence/playground/rosette/rosette/solver/smt/server.rkt:58:0: server-start
   /Users/florence/playground/rosette/rosette/solver/smt/z3.rkt:60:3: solver-clear
   /Users/florence/playground/rosette/rosette/query/core.rkt:54:0: ∃-solve
   /Users/florence/work/circuitous/circuitous-lib/circuitous/private/interp-unit.rkt:230:2: verify/f
   /Users/florence/playground/rosette/rosette/base/core/bool.rkt:319:0: evaluate-with-asserts
   /Users/florence/work/circuitous/circuitous-lib/circuitous/private/interp-unit.rkt:117:2: verify-same/multi
   /Users/florence/work/circuitous/circuitous-lib/circuitous/interp.rkt:103:0: assert-same
   eval-one-top
   /Applications/Racket/racket/share/pkgs/sandbox-lib/racket/sandbox.rkt:510:0: call-with-custodian-shutdown
   /Applications/Racket/racket/collects/racket/private/more-scheme.rkt:148:2: call-with-break-parameterization

By [S-consistent] we know that S is true. 

In [4]:
(assert-same
 #:constraints 
 '(and 
   ;; by [S-consistent]
   S (and 
   ;; by activation condition
   (implies (not (or (and GO S) (and RES p_SEL)))
            (and (not p_K0) (not p_Kn))) (and
   (implies (not (or (and GO (not S)) (and RES q_SEL)))
            (and (not q_K0) (not q_Kn))) (and 
   ;; by [sel-start]
   (not q_SEL) (and 
   ;; by [sel-defined]
   (or p_SEL (not p_SEL))
   (or q_SEL (not q_SEL)))))))
 start p)

(define add-S
  (replace 
   #:constraints 'S
   start
   '[S true]))

subprocess: `execute' access denied for /Users/florence/playground/rosette/rosette/solver/smt/../../../bin/z3
  context...:
   /Users/florence/playground/rosette/rosette/solver/smt/server.rkt:58:0: server-start
   /Users/florence/playground/rosette/rosette/solver/smt/z3.rkt:60:3: solver-clear
   /Users/florence/playground/rosette/rosette/query/core.rkt:54:0: ∃-solve
   /Users/florence/work/circuitous/circuitous-lib/circuitous/private/interp-unit.rkt:230:2: verify/f
   /Users/florence/playground/rosette/rosette/base/core/bool.rkt:319:0: evaluate-with-asserts
   /Users/florence/work/circuitous/circuitous-lib/circuitous/private/interp-unit.rkt:117:2: verify-same/multi
   /Users/florence/work/circuitous/circuitous-lib/circuitous/interp.rkt:103:0: assert-same
   eval-one-top
   /Applications/Racket/racket/share/pkgs/sandbox-lib/racket/sandbox.rkt:693:3
   /Applications/Racket/racket/share/pkgs/sandbox-lib/racket/sandbox.rkt:510:0: call-with-custodian-shutdown
   /Applications/Racket/racket/